In [1]:
import os
import sys
import numpy as np
import pandas as pd
#from scipy.stats import norm

import pybedtools
import re
#import matplotlib as plt
import pyBigWig as pbw
import time
import urllib.request
import matplotlib.pyplot as plt
import json
import requests

In [15]:
# Prepare SNP input files for CHEERS:
#Study_table=pd.read_json("/home/xg1/genetics-v2d-data/tmp/210601/merged_study_table.json",orient='records', 
#                         lines=True)

Study_table=pd.read_parquet("/Users/xg1/Downloads/v2d_210601_studies.parquet")
Study_table_sumstats=Study_table.loc[Study_table['has_sumstats'] == True]

Study_table.head()

#Bowels_study_table=Study_table_sumstats.loc[Study_table_sumstats['trait_reported'].str.contains(
#        'dermatitis', flags=re.I, regex=True) & (Study_table_sumstats['num_assoc_loci'] > 0)]
Bowels_study_table=Study_table_sumstats.loc[Study_table_sumstats['study_id'].str.contains(
        'FINNGEN_R5_L12_ATOPIC', flags=re.I, regex=True) & (Study_table_sumstats['num_assoc_loci'] > 10)]
print(Bowels_study_table)

                   study_id pmid    pub_date pub_journal pub_title  \
1842  FINNGEN_R5_L12_ATOPIC       2021-05-11                         

      pub_author     trait_reported     trait_efos   ancestry_initial  \
1842  FINNGEN_R5  Atopic dermatitis  [EFO_0000274]  [European=205764]   

     ancestry_replication  n_initial  n_replication  n_cases trait_category  \
1842                   []     205764            0.0   7024.0  Immune system   

      num_assoc_loci  has_sumstats  
1842              19          True  


In [ ]:
study_table=pd.read_parquet("/home/xg1/genetics-v2d-data/output/210601/studies.parquet")
Bowels_study_table=study_table.loc[study_table['trait_reported'].str.contains(
        'rheumatoid arthritis', flags=re.I, regex=True)]
print(Bowels_study_table.to_string())

In [53]:
Cred_SNPs=pd.read_parquet("/Users/xg1/Downloads/finemapping.parquet",
                engine='pyarrow')

Select_SNPs=Cred_SNPs.loc[Cred_SNPs['study_id'].str.contains("GCST005413")].reset_index()
Select_SNPs=Select_SNPs[['tag_chrom', 'tag_pos']]
Select_SNPs['tag_chrom']="chr"+Select_SNPs['tag_chrom']
print(Select_SNPs)

    tag_chrom   tag_pos
0        chr1  39570256
1        chr1  39371430
2        chr1  39382675
3        chr1  39402949
4        chr1  39375096
..        ...       ...
495      chr9  22133985
496      chr9  22134173
497      chr9  22134652
498      chr9  22133646
499      chr9  22136441

[500 rows x 2 columns]


In [49]:
Cred_SNPs=pd.read_parquet("/Users/xg1/Downloads/finemapping.parquet",
                engine='pyarrow')

Select_SNPs=Cred_SNPs.loc[Cred_SNPs['study_id'].str.contains("GCST005413")].reset_index()
Select_SNPs=Select_SNPs[['tag_chrom', 'tag_pos']]
Select_SNPs['tag_chrom']="chr"+Select_SNPs['tag_chrom']
print(Select_SNPs)

lo=LiftOver("hg19", "hg38")

def Call_liftover(input_chr, input_pos, input_LO):
    return(input_LO.convert_coordinate(input_chr, input_pos))

def Liftover_chr(input_chr, input_pos, input_LO):
    lifted_result=input_LO.convert_coordinate(input_chr, input_pos)
    if(len(lifted_result) == 0):
        return("NA")
    else:
        return(lifted_result[0][0])

def Liftover_pos(input_chr, input_pos, input_LO):
    lifted_result=input_LO.convert_coordinate(input_chr, input_pos)
    if(len(lifted_result) == 0):
        return("NA")
    else:
        return(lifted_result[0][1])
    

Select_SNPs['hg38_chr']=Select_SNPs.apply(lambda x: Liftover_chr(input_chr=x['tag_chrom'], input_pos=x['tag_pos'], input_LO=lo), axis=1)
Select_SNPs['hg38_pos']=Select_SNPs.apply(lambda x: Liftover_pos(input_chr=x['tag_chrom'], input_pos=x['tag_pos'], input_LO=lo), axis=1)
#Select_SNPs=Select_SNPs.loc[-Select_SNPs['hg38_chr'].str.contains("NA")]

Select_SNPs['hg38_names']=Select_SNPs.apply(lambda x: x['hg38_chr']+"_"+str(x['hg38_pos']), axis=1)
Select_SNPs=Select_SNPs[['hg38_names', 'hg38_chr', 'hg38_pos']]


#Select_SNPs['name']=Select_SNPs.apply(lambda x: x['tag_chrom']+"_"+str(x['tag_pos']), axis=1)
#Select_SNPs=Select_SNPs[['name', 'tag_chrom', 'tag_pos']]

print(Select_SNPs)
Select_SNPs.to_csv("/Users/xg1/Downloads/CHEERS/GWAS_inputs/FINNGEN_R5_L12_ATOPIC_hg38.txt", sep="\t", index=False, header=False)

#Select_SNPs.to_csv("/Users/xg1/Downloads/CHEERS/GWAS_inputs/FINNGEN_R5_L12_ATOPIC.txt", sep="\t", index=False, header=False)
#Select_SNPs.
#Traits_all_SNPs_pp['name']=Traits_all_SNPs_pp.apply(lambda x: x['name']+"_"+str(x['pos']), axis=1)




         hg38_names hg38_chr   hg38_pos
0    chr1_119435581     chr1  119435581
1    chr1_119677108     chr1  119677108
2    chr1_118290771     chr1  118290771
3    chr1_119210549     chr1  119210549
4    chr1_117921223     chr1  117921223
..              ...      ...        ...
494   chr8_79470034     chr8   79470034
495   chr8_79470232     chr8   79470232
496   chr8_79500005     chr8   79500005
497   chr8_79440676     chr8   79440676
498  chr9_134714145     chr9  134714145

[458 rows x 3 columns]


In [52]:
lo=LiftOver("hg38", "hg19")

def Generate_hg38_credset(study_ID, trait_name, output_path="./", All_cred=Cred_SNPs):
    Select_SNPs=Cred_SNPs.loc[Cred_SNPs['study_id'].str.contains(study_ID)].reset_index()
    Select_SNPs=Select_SNPs[['tag_chrom', 'tag_pos']]
    Select_SNPs['tag_chrom']="chr"+Select_SNPs['tag_chrom']
    Select_SNPs['hg38_chr']=Select_SNPs.apply(lambda x: Liftover_chr(input_chr=x['tag_chrom'], input_pos=x['tag_pos'], input_LO=lo), axis=1)
    Select_SNPs['hg38_pos']=Select_SNPs.apply(lambda x: Liftover_pos(input_chr=x['tag_chrom'], input_pos=x['tag_pos'], input_LO=lo), axis=1)
    Select_SNPs=Select_SNPs.loc[-Select_SNPs['hg38_chr'].str.contains("NA")]

    Select_SNPs['hg38_names']=Select_SNPs.apply(lambda x: x['hg38_chr']+"_"+str(x['hg38_pos']), axis=1)
    Select_SNPs=Select_SNPs[['hg38_names', 'hg38_chr', 'hg38_pos']]

    Select_SNPs.to_csv(output_path+study_ID+"_"+trait_name+"_"+"hg19.txt", sep="\t", index=False, header=False)


Generate_hg38_credset(study_ID="FINNGEN_R5_L12_ATOPIC", trait_name="", output_path="/Users/xg1/Downloads/CHEERS/GWAS_inputs/")
Generate_hg38_credset(study_ID="GCST002245", trait_name="Alz", output_path="/Users/xg1/Downloads/CHEERS/GWAS_inputs/")
Generate_hg38_credset(study_ID="GCST004131", trait_name="IBD", output_path="/Users/xg1/Downloads/CHEERS/GWAS_inputs/")
Generate_hg38_credset(study_ID="GCST005413", trait_name="T2D", output_path="/Users/xg1/Downloads/CHEERS/GWAS_inputs/")
Generate_hg38_credset(study_ID="GCST005536", trait_name="T1D", output_path="/Users/xg1/Downloads/CHEERS/GWAS_inputs/")
Generate_hg38_credset(study_ID="GCST005569", trait_name="RA", output_path="/Users/xg1/Downloads/CHEERS/GWAS_inputs/")


In [29]:
Select_SNPs.iloc[1]['tag_chrom']

'chr1'

In [31]:
from pyliftover import LiftOver
lo=LiftOver("hg19", "hg38")
lo.convert_coordinate(Select_SNPs.iloc[3]['tag_chrom'], Select_SNPs.iloc[3]['tag_pos'])


[('chr1', 119210549, '+', 20851231461)]

In [42]:
lo=LiftOver("hg19", "hg38")

def Call_liftover(input_chr, input_pos, input_LO):
    return(input_LO.convert_coordinate(input_chr, input_pos))

def Liftover_chr(input_chr, input_pos, input_LO):
    lifted_result=input_LO.convert_coordinate(input_chr, input_pos)
    if(len(lifted_result) == 0):
        return("NA")
    else:
        return(lifted_result[0][0])

def Liftover_pos(input_chr, input_pos, input_LO):
    lifted_result=input_LO.convert_coordinate(input_chr, input_pos)
    if(len(lifted_result) == 0):
        return("NA")
    else:
        return(lifted_result[0][1])
    


In [11]:
from collections import OrderedDict

def variant_info_query(gene_id):
  #construct query string and declare variables that will be sent in query
  api_query = """
    query variantInfo($geneId: String!) {
      variantInfo(variantId: $geneId) {
        rsId
      }
    }
  """ 

  #set base_url for Open Targets Genetics Portal API
  base_url = "https://api.genetics.opentargets.org/graphql"

  #set variables object
  variables = {"geneId": gene_id}

  #perform API call using query string and variables object 
  r = requests.post(base_url, json={"query": api_query, "variables": variables})

  #check status code of GraphQL API response and print error message if code == 400
  if str(r.status_code) == "400":
    print(f"{gene_id} query status code: {r.status_code}")
  else:
    pass

  #transform API response into JSON 
  api_response_as_json = json.loads(r.text)
  
  #print first element of JSON response data
 # return(api_response_as_json["data"]["variantInfo"]['rsId'])
  if(len(api_response_as_json) == 1):
    return(api_response_as_json["data"]["variantInfo"]['rsId'])
  else:
    return("NA")
#variant_info_query("1_1182895_C_T")

def make_query_inputs(chrom, pos, ref, alt):
    input_var=str(chrom)+"_"+str(pos)+"_"+str(ref)+"_"+str(alt)
    return(variant_info_query(input_var))

def Make_LDSC_sumstats(input_sumstats, out_dir, out_prefix):
    raw_sumstats=pd.read_parquet(input_sumstats)
    raw_sumstats['SNP']=raw_sumstats.apply(lambda x: make_query_inputs(x['chrom'], x['pos'], x['ref'], x['alt']), axis=1)
    cols = OrderedDict([
        ('SNP', 'snpid'),
        ('ref', 'a1'),
        ('alt', 'a2'),
        ('n_total', 'N'),
        ('pval', 'P'),
        ('eaf', 'Freq.Allele1')
        ])
    out_sumstats=raw_sumstats.loc[:, list(cols.keys())].rename(columns=cols)
    out_sumstats.to_csv(out_dir+out_prefix+"_sumstats.txt", sep="\t", index=False)

    

#IBD_sumstats['SNP']=IBD_sumstats.apply(lambda x: make_query_inputs(x['chrom'], x['pos'], x['ref'], x['alt']), axis=1)

In [ ]:
raw_sumstats=pd.read_parquet("/Users/xg1/Downloads/ldsc/input_sumstats/GCST005569.parquet")
print(raw_sumstats)

In [ ]:
Make_LDSC_sumstats("/Users/xg1/Downloads/ldsc/input_sumstats/GCST005413.parquet", "/Users/xg1/Downloads/ldsc/Formatted_sumstats/", "T2D")

In [13]:
start_time=time.time()
Make_LDSC_sumstats("/Users/xg1/Downloads/ldsc/input_sumstats/GCST002245.parquet", "/Users/xg1/Downloads/ldsc/Formatted_sumstats/", "Alzheimers")
Make_LDSC_sumstats("/Users/xg1/Downloads/ldsc/input_sumstats/FINNGEN_R5_L12_ATOPIC.parquet", "/Users/xg1/Downloads/ldsc/Formatted_sumstats/", "Atopic")

Make_LDSC_sumstats("/Users/xg1/Downloads/ldsc/input_sumstats/GCST005413.parquet", "/Users/xg1/Downloads/ldsc/Formatted_sumstats/", "T2D")
Make_LDSC_sumstats("/Users/xg1/Downloads/ldsc/input_sumstats/GCST005536.parquet", "/Users/xg1/Downloads/ldsc/Formatted_sumstats/", "T1D")
#Make_LDSC_sumstats("/Users/xg1/Downloads/ldsc/input_sumstats/GCST005569.parquet", "/Users/xg1/Downloads/ldsc/Formatted_sumstats/", "RA")
Make_LDSC_sumstats("/Users/xg1/Downloads/ldsc/input_sumstats/GCST004131.parquet", "/Users/xg1/Downloads/ldsc/Formatted_sumstats/", "IBD")
print(time.time()-start_time)

JSONDecodeError: ('Expecting value: line 2 column 1 (char 1)', 'occurred at index 2467')

In [20]:
Sumstat_annot_dir="/Users/xg1/Downloads/ldsc/Comparison_baseline/"
#print(os.listdir(Sumstat_annot_dir))
Test_samples=["BSS00121", "BSS00122", "BSS00123", "BSS00206", "BSS00207", "BSS01712", "BSS01714", "BSS00196", "BSS00197", "BSS00188", "BSS00189", "BSS00233", "BSS00234", "BSS00227", "BSS00228", "BSS01113", "BSS01120", "BSS00084", "BSS00330", "BSS00079", "BSS00080", "BSS01665", "BSS01666", "BSS01169", "BSS00511", "BSS01068", "BSS01071"]
for i in range(1, 23):
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [3]:
Cred_SNPs=pd.read_parquet("/home/xg1/genetics-v2d-data/output/210601/finemapping.parquet",
                engine='pyarrow')

In [14]:
Cred_SNPs.loc[Cred_SNPs['study_id'].str.contains("GCST000612")]

,study_id,lead_chrom,lead_pos,lead_ref,lead_alt,tag_chrom,tag_pos,tag_ref,tag_alt,log10_ABF,posterior_prob
256932,GCST000612,1,192567683,C,A,1,192567683,C,A,22.584086,0.996563
628144,GCST000612,11,128516011,T,C,11,128516011,T,C,19.382221,0.526447
628145,GCST000612,11,128516011,T,C,11,128511079,C,T,19.256326,0.464173
771037,GCST000612,12,111569952,C,T,12,111569952,C,T,25.660509,0.606931
771039,GCST000612,12,111569952,C,T,12,111446804,T,C,25.226076,0.393068
1620649,GCST000612,2,102454108,T,C,2,102454108,T,C,27.880535,0.978589
1691583,GCST000612,2,181143073,T,C,2,181143073,T,C,15.014774,0.775451
1691585,GCST000612,2,181143073,T,C,2,181131318,A,G,13.410425,0.155881
1691587,GCST000612,2,181143073,T,C,2,181105204,T,G,11.841626,0.032469
1984428,GCST000612,3,46193709,C,T,3,46193709,C,T,19.702220,0.597258


In [27]:
Input_SNPs=pd.read_csv("/home/xg1/CHEERS/input_SNPs/Filtered_GCST004131_hg38.nopp.input.bed",
                      sep='\t', names=["name", "chr", "pos"])
Input_SNPs.head()

,name,chr,pos
0,chr1,chr1,8108201
1,chr1,chr1,8092042
2,chr1,chr1,8120150
3,chr1,chr1,8109690
4,chr1,chr1,8108933


In [31]:
#Input_SNPs['name']=Input_SNPs['name']+"_"+str(Input_SNPs['pos'])
abc=Input_SNPs.apply(lambda x: x['name']+"_"+str(x['pos']), axis=1)
#Input_SNPs.apply(lambda x: print(x))

In [35]:
Input_SNPs['name']=abc

In [37]:
Input_SNPs.to_csv("/home/xg1/CHEERS/input_SNPs/Correct_IBD_SNPs.txt",
                 sep="\t", header=False, index=False)

In [42]:
Cred_SNPs.shape

(3218820, 11)

In [68]:
## PICS output is in ld.parquet
PICS_SNPs=pd.read_parquet("/home/xg1/genetics-v2d-data/output/210601/ld.parquet/")

## sumstat credsets in finemapping.parquet:
Cred_SNPs=pd.read_parquet("/Users/xg1/Downloads/finemapping.parquet",
                engine='pyarrow')

## load up study table and search for studies which match a EFO
study_table=pd.read_parquet("/home/xg1/genetics-v2d-data/output/210601/studies.parquet")

Trait_matched_studies=study_table.loc[study_table.trait_efos.apply(lambda x: "EFO_0000685" in x)]

Trait_matched_sumstats=Trait_matched_studies.loc[Trait_matched_studies['has_sumstats'] == True]['study_id'].unique()


Trait_matched_PICS=Trait_matched_studies.loc[Trait_matched_studies['has_sumstats'] == False]['study_id'].unique()


## For studies which have sumstats, look up credible SNPs in finemapping.parquet

Trait_sumstat_SNPs=Cred_SNPs.loc[Cred_SNPs['study_id'].isin(Trait_matched_sumstats)]
Trait_sumstat_SNPs.drop(['log10_ABF'], axis=1)

## Otherwise get credible SNPs from ld.parquet

Trait_PICS_SNPs=PICS_SNPs.loc[PICS_SNPs['study_id'].isin(Trait_matched_PICS)]
Trait_PICS_SNPs=Trait_PICS_SNPs.drop(['overall_r2', 'pics_mu', 'pics_95perc_credset', 'pics_99perc_credset', 'AFR_1000G_prop',
                      'AMR_1000G_prop', 'EAS_1000G_prop', 'EUR_1000G_prop', 'SAS_1000G_prop' ], axis=1)
Trait_PICS_SNPs=Trait_PICS_SNPs.rename(columns={'pics_postprob':'posterior_prob'})
## Merge the SNPs retrieved together, keep unique credible SNPs

Trait_all_SNPs=pd.concat([Trait_sumstat_SNPs, Trait_PICS_SNPs])

## Assign names to each row (Chr+pos as a string is fine)

Trait_all_SNPs_pp=Trait_all_SNPs[['tag_chrom', 'tag_pos', 'posterior_prob']]

Traits_all_SNPs_pp['name']=Traits_all_SNPs_pp.apply(lambda x: x['name']+"_"+str(x['pos']), axis=1)


## Output




In [ ]:
Trait_all_SNPs_pp=Trait_all_SNPs[['tag_chrom', 'tag_pos', 'posterior_prob']]

Traits_all_SNPs_pp['name']=Traits_all_SNPs_pp.apply(lambda x: x['tag_chrom']+"_"+str(x['tag_pos']), axis=1)


In [53]:
Trait_sumstat_SNPs.drop(['log10_ABF'], axis=1)


,study_id,lead_chrom,lead_pos,lead_ref,lead_alt,tag_chrom,tag_pos,tag_ref,tag_alt,posterior_prob
31213,GCST005569,1,17347613,C,T,1,17347613,C,T,0.214298
31214,GCST005569,1,17347613,C,T,1,17346235,G,A,0.213230
31215,GCST005569,1,17347613,C,T,1,17348042,C,A,0.145258
31216,GCST005569,1,17347613,C,T,1,17347690,C,T,0.132471
31217,GCST005569,1,17347613,C,T,1,17330628,A,G,0.096870
...,...,...,...,...,...,...,...,...,...,...
3063173,FINNGEN_R5_M13_RHEUMA,8,128503195,T,C,8,128703972,T,C,0.001159
3063174,FINNGEN_R5_M13_RHEUMA,8,128503195,T,C,8,128709557,A,C,0.001077
3110695,GCST005569,9,34710263,G,A,9,34710263,G,A,0.461193
3110696,GCST005569,9,34710263,G,A,9,34710341,G,A,0.263071


In [61]:
Trait_all_SNPs=pd.concat([Trait_sumstat_SNPs.drop(['log10_ABF'], axis=1)
, Trait_PICS_SNPs.drop(['overall_r2', 'pics_mu', 'pics_95perc_credset', 'pics_99perc_credset', 'AFR_1000G_prop',
                      'AMR_1000G_prop', 'EAS_1000G_prop', 'EUR_1000G_prop', 'SAS_1000G_prop' ], axis=1)])


In [62]:
Trait_all_SNPs

,study_id,lead_chrom,lead_pos,lead_ref,lead_alt,tag_chrom,tag_pos,tag_ref,tag_alt,posterior_prob,pics_postprob
31213,GCST005569,1,17347613,C,T,1,17347613,C,T,0.214298,NaN
31214,GCST005569,1,17347613,C,T,1,17346235,G,A,0.213230,NaN
31215,GCST005569,1,17347613,C,T,1,17348042,C,A,0.145258,NaN
31216,GCST005569,1,17347613,C,T,1,17347690,C,T,0.132471,NaN
31217,GCST005569,1,17347613,C,T,1,17330628,A,G,0.096870,NaN
...,...,...,...,...,...,...,...,...,...,...,...
12152701,GCST90016610,2,60841032,C,T,2,61026898,T,C,NaN,0.002549
12152702,GCST90016610,2,60841032,C,T,2,61028575,C,T,NaN,0.002549
12152703,GCST90016610,2,60841032,C,T,2,60791088,C,CA,NaN,0.002530
12152704,GCST90016610,2,60841032,C,T,2,61052261,G,A,NaN,0.002332
